In [7]:
using DelaySSAToolkit
using Catalyst
using Statistics
using Test
using BenchmarkTools
using DataFrames
using CSV

In [13]:
using Catalyst
using Latexify
using KaTeX

ArgumentError: ArgumentError: Package KaTeX not found in current path:
- Run `import Pkg; Pkg.add("KaTeX")` to install the KaTeX package.


0 $\rightarrow$ $A_1$, $\rightarrow$ $A_2$, ..., $\rightarrow$ $A_5$ $\rightarrow$ 0 with delay 1 for each cascade delay

In [12]:
rn = @reaction_network begin
    5, 0 --> A1
end
latexify(rn)

L"\begin{align}
\require{mhchem}
\ce{ \varnothing &->[5] A1}
\end{align}
"

In [14]:
delay_trigger_affect! = []
chain_len = 10
delay_trigger_affect! = function (integrator, rng)
    append!(integrator.de_chan[1], 1.)
end

#11 (generic function with 1 method)

In [15]:
u0 = zeros(chain_len)
de_chan0 = []
for _ in 1:chain_len
    push!(de_chan0, [])
end
tspan = (0.,50.)
delay_complete_affect! = []
for i in 1:chain_len-1
    push!(delay_complete_affect!, function (integrator, rng)
        integrator.u[i] -= 1 # A_prev minus 1
        integrator.u[i+1] += 1 # A plus 1
        append!(integrator.de_chan[i+1], 1.) # add to the delay channel
    end
    )
end
push!(delay_complete_affect!, function (integrator, rng)
    integrator.u[chain_len] -= 1 # A_prev minus 1
end)

10-element Vector{Any}:
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #13 (generic function with 1 method)
 #15 (generic function with 1 method)

In [16]:
delay_trigger = Dict(Pair(1, delay_trigger_affect!))
delay_complete = Dict(i=>delay_complete_affect![i] for i in 1:chain_len)
delay_interrupt = Dict()

Dict{Any, Any}()

In [17]:
delayjumpset = DelayJumpSet(delay_trigger, delay_complete, delay_interrupt)
jumpsys = convert(JumpSystem, rn, combinatoric_ratelaws = false)
dprob = DiscreteProblem(jumpsys, u0, tspan)

DiscreteProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 50.0)
u0: 10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [18]:
algo_list =[DelayDirect(),DelayRejection(),DelayMNRM(),DelayDirectCR()]

4-element Vector{DelaySSAToolkit.AbstractDelayAggregatorAlgorithm}:
 DelayDirect()
 DelayRejection()
 DelayMNRM()
 DelayDirectCR()

In [21]:
djprob = DelayJumpProblem(jumpsys,dprob,DelayDirect(),delayjumpset, de_chan0, save_positions=(false,false))


Number of constant rate jumps: 0
Number of variable rate jumps: 0
Have a mass action jump
Number of delay trigger reactions: 1
Number of delay interrupt reactions: 0


In [24]:
medianlist=[]
for algo in algo_list
    djprob = DelayJumpProblem(jumpsys,dprob,algo,delayjumpset, de_chan0, save_positions=(false,false))
    a=@benchmark solve(djprob, SSAStepper())
    push!(medianlist,copy(median(a).time/1e9))
    print(djprob.aggregator," OK","\n")
end

DelayDirect() OK


DelayRejection() OK


DelayMNRM() OK


DelayDirectCR() OK


In [3]:
medianlist

UndefVarError: UndefVarError: medianlist not defined

In [26]:
using Plots
medianvalue=[string(round(mt,digits=6),"s") for mt in medianlist]
algo_name = ["DelayDirect","DelayRejection","DelayMNRM","DelayDirectCR"]
p1=bar(algo_name,medianlist,legend=:false,title="cascade",ylabel="mediantime")
scatter!(algo_name, 0.0005 .+ medianlist , markeralpha=0, series_annotations=medianvalue)
savefig(p1,"test/zxytest/results/cascade_median.png")

In [1]:
using Plots

In [2]:
medianvalue=[string(round(mt,digits=6),"s") for mt in medianlist]
algo_name = ["DelayDirect","DelayRejection","DelayMNRM","DelayDirectCR"]

UndefVarError: UndefVarError: medianlist not defined